In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
raw_data = pd.read_csv('./data/online-test-dataset.csv')
print('Number of rows:', len(raw_data))
raw_data.head()

Number of rows: 858220


,date,store,item,sales
0,1/1/13,1,1,13.0
1,1/2/13,1,1,11.0
2,1/3/13,1,1,14.0
3,1/4/13,1,1,13.0
4,1/5/13,1,1,10.0
